In [1]:
# importing libraries
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [11]:
import pickle

# Replace 'your_file.pkl' with the actual path to your Pickle file
file_path = 'texts.pkl'

# Open the Pickle file in binary read mode
with open(file_path, 'rb') as file:
    try:
        # Load the object from the Pickle file
        your_object = pickle.load(file)

        # Print information about the loaded object
        print("Type of the loaded object:", type(your_object))

        # If it's a dictionary, print keys
        if isinstance(your_object, dict):
            print("Keys in the dictionary:", list(your_object.keys()))

        # You can add more type-specific checks or print statements based on the expected structure

    except Exception as e:
        print("Error loading the Pickle file:", e)


Type of the loaded object: <class 'list'>


In [10]:
import h5py

# Replace 'your_file.h5' with the actual path to your HDF5 file
file_path = 'model.h5'

# Open the HDF5 file in read-only mode
with h5py.File(file_path, 'r') as file:
    # Print keys at the root level
    print("Keys at the root level:", list(file.keys()))

    # Iterate through keys and print information about groups and datasets
    for key in file.keys():
        if isinstance(file[key], h5py.Group):
            print(f"Group: {key}")
            # Print keys within the group
            print(f"  Keys in {key}: {list(file[key].keys())}")
        elif isinstance(file[key], h5py.Dataset):
            print(f"Dataset: {key}")
            # Print information about the dataset
            print(f"  Shape: {file[key].shape}")
            print(f"  Datatype: {file[key].dtype}")
            # You can choose to print a subset of the data if the dataset is not too large
            print(f"  Data (partial): {file[key][:]}")
        else:
            print(f"Unknown type for key: {key}")

# The file is automatically closed when you exit the 'with' block


Keys at the root level: ['model_weights', 'optimizer_weights']
Group: model_weights
  Keys in model_weights: ['dense', 'dense_1', 'dense_2', 'dropout', 'dropout_1', 'top_level_model_weights']
Group: optimizer_weights
  Keys in optimizer_weights: ['SGD']


In [2]:
# tokenizing and lematizing
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

286 documents
129 classes ['What are the types of depression?', 'about', 'afternoon', 'anxious', 'ask', 'at what age does anxiety peak?', 'can lack of sleep make you feel sad?', 'can low blood sugar cause suicidal thoughts?', 'casual', 'creation', 'death', 'default', 'depressed', 'do we control our thoughts?', 'does oversleeping cause depression?', 'done', 'evening', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'friends', 'goodbye', 'greeting', 'happy', 'hate-me', 'hate-you', 'help', 'how can we reduce anxiety?', 'how does depression affect the world?', 'how long can anxiety last?', 'how many thoughts a day do we have?', 'i am a victim of bullying', 'i am afraid i will fail again', 'i am afraid to file 

In [3]:

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [4]:

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (286, 2) + inhomogeneous part.

In [5]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


NameError: name 'train_x' is not defined

In [ ]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [6]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)
print("model created")

NameError: name 'train_x' is not defined